In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


files = [
    "results_breast.csv",
    "results_prostate.csv",
    "results_renal.csv",
    "results_thoat.csv"
]

df_results = pd.concat([pd.read_csv(f"results/multi_classifiers/{f}") for f in files])
df_results['model'] = df_results.classifier.apply(lambda s: s[:s.find(' (')])
df_results['params'] = df_results.classifier.apply(lambda s: s[s.find(' ('):])
df_results.head()

In [ ]:
# Table with average stability values for the classifiers and datasets tested

df_stability = pd.DataFrame(columns=np.unique(df_results.dataset))
for model in np.unique(df_results.model):
    mean_stabilities = [df_results[(df_results.model==model) & (df_results.dataset==dataset)].Lustgarten.mean() for dataset in df_stability.columns]
    df_stability.loc[model] = mean_stabilities

df_stability

In [ ]:
# Table with average test accuracy values for tested classifiers and datasets

df_acc = pd.DataFrame(columns=np.unique(df_results.dataset))
for model in np.unique(df_results.model):
    mean_acc = [df_results[(df_results.model==model) & (df_results.dataset==dataset)].acc_test.mean() for dataset in df_acc.columns]
    df_acc.loc[model] = mean_acc

df_acc

In [ ]:
# Harmonic mean stability and test accuracy

df_harm_mean = pd.DataFrame(columns=np.unique(df_results.dataset))
values = 2/(1/df_acc.values + 1/df_stability.values)
for model,v in zip(np.unique(df_results.model), values):
    df_harm_mean.loc[model] = v

df_harm_mean

In [ ]:
# Generate graphs of the dependence of the parameters A, B, C, D on the value of p
# A - Lustgarten features selection stability
# B - Nogueira features selection stability
# C - Average number of selected features
# D - Test accuracy

datasets_num = len(df_results['dataset'].unique())
fig, axes = plt.subplots(datasets_num, 4, figsize=(16, 4*datasets_num))

for i, (dataset, df_dataset) in enumerate(df_results.groupby(['dataset'])):
    ax = axes[i]
    for model_name, df in df_dataset.groupby(['model']):
        label = f"{model_name}"
        ax[0].plot(df.p, df['Lustgarten'], label=label)
        ax[1].plot(df.p, df['Nogueira'], label=label)
        ax[2].plot(df.p, df['#features_mean'], label=label)
        ax[3].plot(df.p, df['acc_test'], label=label)
        
        ax[0].set_ylabel(f"{dataset}\n", fontsize=16)
        if i==0:
            ax[0].set_title("Lustgarten features selection stability\n", fontsize=12)
            ax[1].set_title("Nogueira features selection stability\n", fontsize=12)
            ax[2].set_title("Average number of selected features\n", fontsize=12)
            ax[3].set_title("Test accuracy\n", fontsize=12)

        ax[0].set_ylim([0, 1.0])
        ax[1].set_ylim([0, 1.0])
        ax[3].set_ylim([0.6, 1.0])
        if dataset == "Breast":
            ax[2].set_ylim([55, 68])
        elif dataset == "Prostate":
            ax[2].set_ylim([20, 32])
        elif dataset == "Renal":
            ax[2].set_ylim([20, 32])
        elif dataset == "Throat":
            ax[2].set_ylim([20, 32])

handles, labels = ax[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center', ncol=4)